# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [38]:
# Part 1 — recompute K (u = -K x) and save control-K-intended.tsv
import pandas as pd, numpy as np
from scipy.linalg import solve_discrete_are

A_df = pd.read_csv("model-A.tsv", sep="\t", index_col=0)
B_df = pd.read_csv("model-B.tsv", sep="\t", index_col=0)
Q_df = pd.read_csv("cost-Q.tsv", sep="\t", index_col=0)
R_df = pd.read_csv("cost-R.tsv", sep="\t", index_col=0)

# Ensure column/row order is [x, v, a]
order = ["x","v","a"]
A_df = A_df.loc[order, order]
Q_df = Q_df.loc[order, order]
# B is a single column (u) indexed by x,v,a
B_df = B_df.loc[order, ["u"]]

A, B, Q, R = A_df.values, B_df.values, Q_df.values, R_df.values
P = solve_discrete_are(A, B, Q, R)
K = np.linalg.solve(R + B.T @ P @ B, B.T @ P @ A)  # u = -K x



Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [39]:
# YOUR CHANGES HERE
pd.DataFrame(K, index=["u"], columns=order).to_csv(
    "control-K-intended.tsv", sep="\t", index=True
)
print("Saved control-K-intended.tsv")

Saved control-K-intended.tsv


Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [40]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

# Load intended K from Part 1
K = pd.read_csv("control-K-intended.tsv", sep="\t", index_col=0).loc["u", ["x","v","a"]].to_numpy()

# Load logs
log = pd.read_csv("qc-log.tsv", sep="\t")

# Build state matrix in [x, v, a] order
X = log[["x","v","a"]].to_numpy(dtype=float)

# LQR: u = -K x
u_check = -(X @ K)

# Create required output with exact columns
out = pd.DataFrame({
    "index": log["index"] if "index" in log.columns else np.arange(len(log)),
    "u_check": u_check
})

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [41]:
# YOUR CHANGES HERE

out.to_csv("qc-check.tsv", sep="\t", index=False)
print("Saved qc-check.tsv")

Saved qc-check.tsv


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [42]:
import pandas as pd, numpy as np

log = pd.read_csv("qc-log.tsv", sep="\t")
u_col = "u" if "u" in log.columns else "u_check"  # accept either

X = log[["x","v","a"]].to_numpy(float)
y = log[u_col].to_numpy(float)

beta, *_ = np.linalg.lstsq(X, y, rcond=None)   # u ≈ X @ beta  (no intercept)


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [43]:
# YOUR CHANGES HERE

K_actual = pd.DataFrame([-beta], index=["u"], columns=["x","v","a"])
K_actual.to_csv("control-K-actual.tsv", sep="\t", index=True)

Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [44]:
import pandas as pd
import numpy as np

# Load & sort (if 'index' exists)
df = pd.read_csv("qc-train.tsv", sep="\t")
if "index" in df.columns:
    df = df.sort_values("index").reset_index(drop=True)

# Build (x_t, u_t) → x_{t+1}
X_t  = df[["x","v","a"]].to_numpy()[:-1, :]
u_t  = df[["u"]].to_numpy()[:-1, :]
Y_tp = df[["x","v","a"]].to_numpy()[1:, :]

# Design matrix: [x_t, v_t, a_t, u_t]
X_design = np.hstack([X_t, u_t])                 # (n-1)×4

# Least-squares: Y_{t+1} ≈ X_design @ Θ
Theta, *_ = np.linalg.lstsq(X_design, Y_tp, rcond=None)  # 4×3
# Rows of Θ: [x, v, a, u]; Cols: [x', v', a']

# Map to A (3×3) and B (3×1)
A_new = Theta[0:3, :].T                          # (3×3)
B_new = Theta[3, :].reshape(3, 1)                # (3×1)

# Labeled DataFrames
A_df = pd.DataFrame(A_new, index=["x","v","a"], columns=["x","v","a"])
B_df = pd.DataFrame(B_new, index=["x","v","a"], columns=["u"])

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [45]:
# Save outputs
A_df.to_csv("model-A-new.tsv", sep="\t", index=True)
B_df.to_csv("model-B-new.tsv", sep="\t", index=True)

# Optional 
Y_hat = X_design @ Theta
rmse = np.sqrt(np.mean((Y_tp - Y_hat)**2, axis=0))
print("Saved model-A-new.tsv and model-B-new.tsv")
print("RMSE for [x', v', a']:", rmse)

Saved model-A-new.tsv and model-B-new.tsv
RMSE for [x', v', a']: [9.48993751e-16 3.37486040e-16 2.55714247e-16]


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [46]:
ack_text = """Acknowledgments

I used ChatGPT as a study partner to clarify  and check errors, and simplify code while completing this project. 


For transparency, I am including links to the transcripts of these interactions:

1. ChatGPT guidance: [https://chatgpt.com/share/68d20a8d-0870-8007-bb79-080efab54c33]
These resources document the support I received and ensure compliance with the class generative AI policy.
"""

with open("acknowledgments.txt","w") as f:
    f.write(ack_text)
